# 1. PREPROCESSING

## 1.1 Imports libraries

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer #process null values
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
# system
import itertools
# matplotlib and seaborn for plotting
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import matplotlib.pyplot as plt
#import seaborn as sns

## 1.2 Load datasets

In [2]:
#The question data
question_data = pd.read_csv('data_all/question.csv')
print(question_data.shape)
question_data.head(5)


(129, 2)


,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,Hobby,Do you code as a hobby?
2,OpenSource,Do you contribute to open source projects?
3,Country,In which country do you currently reside?
4,Student,"Are you currently enrolled in a formal, degree..."


In [3]:
#The training data
train_data = pd.read_csv('data_all/train.csv')
#train_data.head(5)

In [4]:
#The test data
test_data = pd.read_csv('data_all/test.csv')
#test_data['SalaryType'] = test_data['SalaryType'].fillna('Yearly')
#test_data.head(5)

In [5]:
#'Student','Employment', 'FormalEducation','UndergradMajor'
option_choices_cols_name= ['Hobby', 'OpenSource',
 'CompanySize', 'JobSatisfaction', 'CareerSatisfaction',
 'HopeFiveYears', 'JobSearchStatus', 'LastNewJob', 'UpdateCV', 'Currency', 'SalaryType',
 'TimeFullyProductive','AgreeDisagree1','AgreeDisagree2', 'AgreeDisagree3',
 'OperatingSystem','NumberMonitors', 'CheckInCode', 'AdBlocker','AdBlockerDisable','AdsAgreeDisagree1',
 'AdsAgreeDisagree2', 'AdsAgreeDisagree3', 'AIDangerous','AIInteresting',
 'AIResponsible', 'AIFuture','EthicsChoice', 'EthicsReport','EthicsResponsible','EthicalImplications',
 'StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount', 'StackOverflowParticipate',
 'StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember', 'HypotheticalTools1',
 'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'HypotheticalTools5','WakeTime','HoursComputer',
 'HoursOutside', 'SkipMeals', 'Exercise', 'EducationParents','Age', 'Dependents',]

# Generate and mapping features by country

In [6]:
#Generate some features by country
Gen_map_country=train_data[train_data['Salary']>0]
Gen_map_country['avg_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('mean')
Gen_map_country['min_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('min')
Gen_map_country['max_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('max')
Gen_map_country['count_sal'] = Gen_map_country['No'].groupby(Gen_map_country['Country']).transform('count')
Gen_map_country['mode_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform(lambda x:x.value_counts().index[0])
Gen_map_country['std_sal']=Gen_map_country.Country.map(Gen_map_country.groupby('Country')['Salary'].std())

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [7]:
#Create map for country
Map_country=Gen_map_country[['Country','avg_sal','min_sal','max_sal','count_sal','mode_sal','std_sal']]
Salmap_country=Map_country.drop_duplicates()
#Salmap

In [8]:
#Mapping new gen features to train, test base on Country
train_data=pd.merge(train_data,Salmap_country,on='Country',how='left')
test_data=pd.merge(test_data,Salmap_country,on='Country',how='left')

# Generate and mapping features by YearsCoding

In [9]:
#Generate some features by YearsCoding
Gen_map_YearsCoding=train_data[train_data['Salary']>0]
Gen_map_YearsCoding['YearsCoding_avg_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('mean')
Gen_map_YearsCoding['YearsCoding_min_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('min')
Gen_map_YearsCoding['YearsCoding_max_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('max')
Gen_map_YearsCoding['YearsCoding_count_sal'] = Gen_map_YearsCoding['No'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('count')
Gen_map_YearsCoding['YearsCoding_std_sal']=Gen_map_YearsCoding.YearsCoding.map(Gen_map_YearsCoding.groupby('YearsCoding')['Salary'].std())
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [10]:
#Create map for year coding
Map_Yearscoding=Gen_map_YearsCoding[['YearsCoding','YearsCoding_avg_sal','YearsCoding_min_sal',
                                     'YearsCoding_max_sal','YearsCoding_count_sal','YearsCoding_std_sal']]
Salmap_Yearscoding=Map_Yearscoding.drop_duplicates()
#Salmap

In [11]:
#Mapping new gen features to train, test base on Yearscoding
train_data=pd.merge(train_data,Salmap_Yearscoding,on='YearsCoding',how='left')
test_data=pd.merge(test_data,Salmap_Yearscoding,on='YearsCoding',how='left')

# Generate some features by RaceEthnicity

In [12]:
#Generate some features by RaceEthnicity
Gen_map_RaceEthnicity=train_data[train_data['Salary']>0]
Gen_map_RaceEthnicity['RaceEthnicity_avg_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('mean')
Gen_map_RaceEthnicity['RaceEthnicity_min_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('min')
Gen_map_RaceEthnicity['RaceEthnicity_max_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('max')
Gen_map_RaceEthnicity['RaceEthnicity_count_sal'] = Gen_map_RaceEthnicity['No'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('count')
Gen_map_RaceEthnicity['RaceEthnicity_std_sal']=Gen_map_RaceEthnicity.RaceEthnicity.map(Gen_map_RaceEthnicity.groupby('RaceEthnicity')['Salary'].std())
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [13]:
#Create map for RaceEthnicity
Map_RaceEthnicity=Gen_map_RaceEthnicity[['RaceEthnicity','RaceEthnicity_avg_sal','RaceEthnicity_min_sal',
                                         'RaceEthnicity_max_sal','RaceEthnicity_count_sal','RaceEthnicity_std_sal']]
Salmap_RaceEthnicity=Map_RaceEthnicity.drop_duplicates()
#Salmap

In [14]:
#Mapping new gen features to train, test base on RaceEthnicity
train_data=pd.merge(train_data,Salmap_RaceEthnicity,on='RaceEthnicity',how='left')
test_data=pd.merge(test_data,Salmap_RaceEthnicity,on='RaceEthnicity',how='left')

# Generate some features by YearsCodingProf

In [15]:
#Generate some features by YearsCodingProf
Gen_map_YearsCodingProf=train_data[train_data['Salary']>0]
Gen_map_YearsCodingProf['YearsCodingProf_avg_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('mean')
Gen_map_YearsCodingProf['YearsCodingProf_min_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('min')
Gen_map_YearsCodingProf['YearsCodingProf_max_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('max')
Gen_map_YearsCodingProf['YearsCodingProf_std_sal']=Gen_map_YearsCodingProf.YearsCodingProf.map(Gen_map_YearsCodingProf.groupby('YearsCodingProf')['Salary'].std())
#Gen_map_YearsCodingProf['YearsCoding_count_sal'] = Gen_map_YearsCodingProf['No'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('count')
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [16]:
#Create map for YearsCodingProf
Map_YearsCodingProf=Gen_map_YearsCodingProf[['YearsCodingProf','YearsCodingProf_avg_sal','YearsCodingProf_min_sal',
                                     'YearsCodingProf_max_sal','YearsCodingProf_std_sal']]
Salmap_YearsCodingProf=Map_YearsCodingProf.drop_duplicates()
#Salmap
#Mapping new gen features to train, test base on YearsCodingProf
train_data=pd.merge(train_data,Salmap_YearsCodingProf,on='YearsCodingProf',how='left')
test_data=pd.merge(test_data,Salmap_YearsCodingProf,on='YearsCodingProf',how='left')

# Generate some features by CurrencySymbol

In [17]:
#Generate some features by CurrencySymbol
Gen_map_CurrencySymbol=train_data[train_data['Salary']>0]
Gen_map_CurrencySymbol['CurrencySymbol_avg_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('mean')
Gen_map_CurrencySymbol['CurrencySymbol_min_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('min')
Gen_map_CurrencySymbol['CurrencySymbol_max_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('max')
Gen_map_CurrencySymbol['CurrencySymbol_std_sal']=Gen_map_CurrencySymbol.CurrencySymbol.map(Gen_map_CurrencySymbol.groupby('CurrencySymbol')['Salary'].std())
#Gen_map_YearsCodingProf['YearsCoding_count_sal'] = Gen_map_YearsCodingProf['No'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('count')
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [18]:
#Create map for YearsCodingProf
Map_CurrencySymbol=Gen_map_CurrencySymbol[['CurrencySymbol','CurrencySymbol_avg_sal','CurrencySymbol_min_sal',
                                     'CurrencySymbol_max_sal','CurrencySymbol_std_sal']]
Salmap_CurrencySymbol=Map_CurrencySymbol.drop_duplicates()
#Salmap
#Mapping new gen features to train, test base on YearsCodingProf
train_data=pd.merge(train_data,Salmap_CurrencySymbol,on='CurrencySymbol',how='left')
test_data=pd.merge(test_data,Salmap_CurrencySymbol,on='CurrencySymbol',how='left')

# Generate some features for all choice columns

In [19]:
for col in option_choices_cols_name:
    print('Start column: '+col)
    name='Gen_map_'+col
    avgname=col+'_avg_sal'
    minname=col+'_min_sal'
    maxname=col+'_max_sal'
    Mapname='Map_'+col
    Salmapname='Salmap_'+col
    name=train_data[train_data['Salary']>0]
    name[avgname] = name['Salary'].groupby(name[col]).transform('mean')
    name[minname] = name['Salary'].groupby(name[col]).transform('min')
    name[maxname] = name['Salary'].groupby(name[col]).transform('max')
    #Create map for YearsCodingProf
    Mapname=name[[col,avgname,minname,maxname]]
    Salmapname=Mapname.drop_duplicates()
    #Salmap
    #Mapping new gen features to train, test base on YearsCodingProf
    train_data=pd.merge(train_data,Salmapname,on=col,how='left')
    test_data=pd.merge(test_data,Salmapname,on=col,how='left')

Start column: Hobby


c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarni

Start column: OpenSource
Start column: CompanySize
Start column: JobSatisfaction
Start column: CareerSatisfaction
Start column: HopeFiveYears
Start column: JobSearchStatus
Start column: LastNewJob
Start column: UpdateCV
Start column: Currency
Start column: SalaryType
Start column: TimeFullyProductive
Start column: AgreeDisagree1
Start column: AgreeDisagree2
Start column: AgreeDisagree3
Start column: OperatingSystem
Start column: NumberMonitors
Start column: CheckInCode
Start column: AdBlocker
Start column: AdBlockerDisable
Start column: AdsAgreeDisagree1
Start column: AdsAgreeDisagree2
Start column: AdsAgreeDisagree3
Start column: AIDangerous
Start column: AIInteresting
Start column: AIResponsible
Start column: AIFuture
Start column: EthicsChoice
Start column: EthicsReport
Start column: EthicsResponsible
Start column: EthicalImplications
Start column: StackOverflowRecommend
Start column: StackOverflowVisit
Start column: StackOverflowHasAccount
Start column: StackOverflowParticipate
Sta

In [20]:
print(test_data.shape)
print(train_data.shape)

(11259, 307)
(33857, 308)


### Target data

In [22]:
target = train_data['Salary'].astype(int)

### Missing values on training data

In [23]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [25]:
missing_values = missing_values_table(train_data)
missing_values.head()

Your selected dataframe has 308 columns.
There are 293 columns that have missing values.


,Missing Values,% of Total Values
TimeAfterBootcamp,30638,90.5
MilitaryUS,25092,74.1
HackathonReasons,20939,61.8
ErgonomicDevices,18628,55.0
AdBlockerReasons,16431,48.5


In [26]:
missing_row_values = train_data.isnull().sum(axis=1)
missing_row_percentage = 100*missing_row_values/(train_data.shape[1]-1)

In [27]:
missing_row_df = pd.concat([missing_row_values,missing_row_percentage],axis=1)

In [28]:
missing_row_df_ren = missing_row_df.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

In [29]:
missing_row_df_ren[missing_row_df_ren.iloc[:,1]!=0].sort_values(
        '% of Total Values', ascending=False).round(1).head(10)

,Missing Values,% of Total Values
15718,241,78.5
29860,239,77.9
23584,238,77.5
21537,236,76.9
15283,236,76.9
13451,235,76.5
30123,235,76.5
2173,234,76.2
23209,234,76.2
22301,234,76.2


## View the data by datatypes

Convert some category columns to numeric

In [30]:
train_data["YearsCoding_convert"] = train_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCoding_convert"] = test_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [31]:
train_data["YearsCodingProf_convert"] = train_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCodingProf_convert"] = test_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [32]:
train_data["StackOverflowRecommend_convert"] = train_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowRecommend_convert"] = test_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [33]:
train_data["StackOverflowJobsRecommend_convert"] = train_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowJobsRecommend_convert"] = test_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [34]:
train_data["HoursComputer_convert"] = train_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})
test_data["HoursComputer_convert"] = test_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})

In [35]:
train_data["HoursOutside_convert"] = train_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})
test_data["HoursOutside_convert"] = test_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})

In [36]:
train_data["Age_convert"] = train_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})
test_data["Age_convert"] = test_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})

In [37]:
train_data["TimeFullyProductive_convert"] = train_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})
test_data["TimeFullyProductive_convert"] = test_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})

In [38]:
train_data["Hobby_convert"] = train_data["Hobby"].map( {'Yes': 1.0,'No': 0.0})
test_data["Hobby_convert"] = test_data["Hobby"].map( {'Yes': 1.0,'No': 0.0})

In [39]:
train_data["OpenSource_convert"] = train_data["OpenSource"].map( {'Yes': 1.0,'No': 0.0})
test_data["OpenSource_convert"] = test_data["OpenSource"].map( {'Yes': 1.0,'No': 0.0})

In [40]:
train_data["Student_convert"] = train_data["Student"].map( {'No': 2.0,'Yes, part-time': 1.0,'Yes, full-time': 0.0})
test_data["Student_convert"] = test_data["Student"].map( {'No': 2.0,'Yes, part-time': 1.0,'Yes, full-time': 0.0})

In [41]:
train_data["FormalEducation_convert"] = train_data["FormalEducation"].map( {'I never completed any formal education': 0.0,
            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1.0,
            'Primary/elementary school': 2.0,
            'Associate degree': 3.0,
            'Some college/university study without earning a degree': 4.0,
            'Bachelor’s degree (BA, BS, B.Eng., etc.)': 5.0,
            'Master’s degree (MA, MS, M.Eng., MBA, etc.)': 6.0,
            'Other doctoral degree (Ph.D, Ed.D., etc.)': 7.0,
            'Professional degree (JD, MD, etc.)': 8.0})
test_data["FormalEducation_convert"] = test_data["FormalEducation"].map( {'I never completed any formal education': 0.0,
            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1.0,
            'Primary/elementary school': 2.0,
            'Associate degree': 3.0,
            'Some college/university study without earning a degree': 4.0,
            'Bachelor’s degree (BA, BS, B.Eng., etc.)': 5.0,
            'Master’s degree (MA, MS, M.Eng., MBA, etc.)': 6.0,
            'Other doctoral degree (Ph.D, Ed.D., etc.)': 7.0,
            'Professional degree (JD, MD, etc.)': 8.0})

In [42]:
train_data["UndergradMajor_convert"] = train_data["UndergradMajor"].map({'Computer science, computer engineering, or software engineering': 2.0,
            'A natural science (ex. biology, chemistry, physics)': 1.0,
            'Web development or web design': 2.0,
            'Information systems, information technology, or system administration': 2.0,
            'A social science (ex. anthropology, psychology, political science)': 0.0,
            'A business discipline (ex. accounting, finance, marketing))': 0.0,
            'Another engineering discipline (ex. civil, electrical, mechanical)': 1.0,
            'A humanities discipline (ex. literature, history, philosophy)': 0.0,
            'Mathematics or statistics)': 2.0,
            'Fine arts or performing arts (ex. graphic design, music, studio art': 1.0,
            'I never declared a major': 0.0,
            'A health science (ex. nursing, pharmacy, radiology)': 0.0})
test_data["UndergradMajor_convert"] = test_data["UndergradMajor"].map({'Computer science, computer engineering, or software engineering': 2.0,
            'A natural science (ex. biology, chemistry, physics)': 1.0,
            'Web development or web design': 2.0,
            'Information systems, information technology, or system administration': 2.0,
            'A social science (ex. anthropology, psychology, political science)': 0.0,
            'A business discipline (ex. accounting, finance, marketing))': 0.0,
            'Another engineering discipline (ex. civil, electrical, mechanical)': 1.0,
            'A humanities discipline (ex. literature, history, philosophy)': 0.0,
            'Mathematics or statistics)': 2.0,
            'Fine arts or performing arts (ex. graphic design, music, studio art': 1.0,
            'I never declared a major': 0.0,
            'A health science (ex. nursing, pharmacy, radiology)': 0.0})

In [43]:
# Number of each type of column
train_data.dtypes.value_counts()

float64    234
object      86
int64        1
dtype: int64

In [49]:
# drop int column type
train_data = train_data.drop('No',axis=1)
test_data = test_data.drop('No',axis=1)

# Data preprocessing

## Clean data

Drop the columns and the rows have more 80% missing values

In [50]:
missing_cols_drop = missing_values[missing_values.iloc[:,1]>80].index

In [51]:
train_data = train_data.drop(missing_cols_drop,axis =1)
test_data = test_data.drop(missing_cols_drop,axis =1)

In [52]:
missing_rows_drop = train_data.isnull().sum(axis=1)

In [53]:
train_data = train_data[missing_rows_drop<(train_data.shape[1]/2)]

In [54]:
print('Training data shape:',train_data.shape)
print('Testing data shape:',test_data.shape)

Training data shape: (33076, 319)
Testing data shape: (11259, 318)


In [55]:
target = train_data['Salary']
train_data = train_data.drop('Salary',axis=1)

## Transform data by datatype

### Float columns

In [56]:
# get columns name
float_cols_name_train = train_data.select_dtypes('float').columns
float_cols_name_test = test_data.select_dtypes('float').columns

In [57]:
#fill na by median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_arr = imputer.fit_transform(train_data.select_dtypes('float'))
test_arr = imputer.transform(test_data.select_dtypes('float'))

In [58]:
float_dataset_train = pd.DataFrame(train_arr,columns=float_cols_name_train)
float_dataset_test = pd.DataFrame(test_arr,columns=float_cols_name_test)

In [59]:
print('Training float dataset shape: ', float_dataset_train.shape)
print('Testing dloat dataset shape: ', float_dataset_test.shape)

Training float dataset shape:  (33076, 233)
Testing dloat dataset shape:  (11259, 233)


In [60]:
float_dataset_train.head()

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert,Hobby_convert,OpenSource_convert,Student_convert,FormalEducation_convert,UndergradMajor_convert
0,9.0,7.0,8.0,3.0,6.0,10.0,2.0,1.0,5.0,4.0,...,10.0,7.0,5.0,40.0,3.0,1.0,0.0,2.0,5.0,2.0
1,9.0,2.0,1.0,6.0,4.0,8.0,3.0,5.0,10.0,7.0,...,10.0,14.0,0.8,40.0,15.0,1.0,1.0,2.0,6.0,1.0
2,9.0,4.0,6.0,1.0,3.0,5.0,2.0,8.0,10.0,7.0,...,10.0,7.0,1.5,40.0,3.0,1.0,1.0,2.0,4.0,2.0
3,7.0,7.0,6.0,4.0,3.0,4.0,6.0,4.0,8.0,6.0,...,10.0,7.0,0.8,30.0,3.0,1.0,0.0,2.0,5.0,2.0
4,3.0,8.0,10.0,2.0,7.0,9.0,1.0,5.0,6.0,4.0,...,10.0,14.0,0.3,30.0,3.0,0.0,1.0,2.0,2.0,2.0


In [61]:
#Check corr
#Checkcorr= float_dataset_train
#Checkcorr['label']=target

In [62]:
# The undersample dataframe
#plt.figure(figsize = (20, 20))
#corr = float_dataset_train.corr()
#sns.heatmap(corr, cmap="coolwarm_r", annot=True)

In [63]:
#corr['label']

### Categorical multiple choice columns

In [66]:
#function for check which column contains multiple-choice
def check_multiple_choices(text):
    if type(text)==str:
        if ';' in text:
            return 1
        else:
            return 0
    return 0

In [67]:
#Categorical columns is columns, which are not in float cols
categorical_cols_name=[col for col in train_data.columns if col not in float_cols_name_train]
#Multiple choices is categorical with character: ";"
mul_choices_cols_name=[]
for i in categorical_cols_name:
    if sum(train_data[i].apply(check_multiple_choices))>0:
        mul_choices_cols_name.append(i)
option_choices_cols_name=[col for col in categorical_cols_name if col not in mul_choices_cols_name]

In [69]:
#list of columns have multiple-choice on train_dataset
#mul_choices_cols_name

In [70]:
#function for count the number of choices on each column
def count_multiple_choice(text):
    if type(text)==str:
        return text.split(";")
    else:
        return []

In [71]:
num_multiple_choice={}
for i in mul_choices_cols_name:
    flat_ls = list(train_data[i].apply(count_multiple_choice))
    num_multiple_choice[i]=len(set(list(itertools.chain(*flat_ls))))

In [72]:
#display the number of choice on columns
#frame_num_choice = pd.DataFrame(num_multiple_choice.items(),columns=['Feature names','Number of choice']).sort_values('Number of choice',ascending=False)
#frame_num_choice

#### Encode multiple-choices to one-hot

In [73]:
#function for encoding multiple-choice to one hot
def encode(ls,ref):
    ini_vec = np.zeros((len(ref)))
    ini_ls=[]
    for i in ls:
        ini_ls.append(ref.index(i))
    ini_vec[ini_ls]=1
    return ini_vec.astype('int')

In [74]:
#apply the function to handle multiple-choices columns
ls_v4_train=[]
ls_v4_test=[]
ls_v2=[]
for c in mul_choices_cols_name:
    
    v1_train = train_data[c].apply(count_multiple_choice) #series
    v1_test = test_data[c].apply(count_multiple_choice) 
    
    v2 = list(set(list(itertools.chain(*v1_train)))) #list of unique of multiple-choices
    v3_train = v1_train.apply(encode,ref=v2) #numpy array
    v3_test = v1_test.apply(encode,ref=v2)
    
    v4_train = np.concatenate(v3_train.values).reshape(-1,len(v2))
    v4_test = np.concatenate(v3_test.values).reshape(-1,len(v2))
    ls_v4_train.append(v4_train)
    ls_v4_test.append(v4_test)
    ls_v2.append(v2)
v5_train = np.concatenate(ls_v4_train,axis=1)
v5_test = np.concatenate(ls_v4_test,axis=1)
v6 = list(itertools.chain(*ls_v2))
multi_choice_dataset_train = pd.DataFrame(v5_train,columns = v6)
multi_choice_dataset_test = pd.DataFrame(v5_test,columns = v6)

In [75]:
multi_choice_dataset_train.head()

,Embedded applications or devices developer,QA or test developer,DevOps specialist,Data scientist or machine learning specialist,Full-stack developer,Product manager,Front-end developer,"C-suite executive (CEO, CTO, etc.)",Game or graphics developer,Educator or academic researcher,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,Middle Eastern,South Asian,East Asian,"Native American, Pacific Islander, or Indigenous Australian"
0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [76]:
multi_choice_dataset_test.head()

,Embedded applications or devices developer,QA or test developer,DevOps specialist,Data scientist or machine learning specialist,Full-stack developer,Product manager,Front-end developer,"C-suite executive (CEO, CTO, etc.)",Game or graphics developer,Educator or academic researcher,...,Bisexual or Queer,Gay or Lesbian,Straight or heterosexual,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,Middle Eastern,South Asian,East Asian,"Native American, Pacific Islander, or Indigenous Australian"
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [77]:
cols=pd.Series(multi_choice_dataset_train.columns)
for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
multi_choice_dataset_train.columns=cols
multi_choice_dataset_test.columns=cols

In [78]:
test_data['DevType'].head(5)

0    Back-end developer;Front-end developer;Full-st...
1    Back-end developer;Database administrator;DevO...
2             Front-end developer;Full-stack developer
3                                   Back-end developer
4    Front-end developer;Full-stack developer;Mobil...
Name: DevType, dtype: object

In [79]:
multi_choice_dataset_train.iloc[:5,:20]

,Embedded applications or devices developer,QA or test developer,DevOps specialist,Data scientist or machine learning specialist,Full-stack developer,Product manager,Front-end developer,"C-suite executive (CEO, CTO, etc.)",Game or graphics developer,Educator or academic researcher,Desktop or enterprise applications developer,System administrator,Student,Back-end developer,Marketing or sales professional,Database administrator,Designer,Data or business analyst,Mobile developer,Engineering manager
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [80]:
multi_choice_dataset_train_generate= multi_choice_dataset_train[cols]
multi_choice_dataset_train_generate['Salary']=target
multi_choice_dataset_train_generate.head()

,Embedded applications or devices developer,QA or test developer,DevOps specialist,Data scientist or machine learning specialist,Full-stack developer,Product manager,Front-end developer,"C-suite executive (CEO, CTO, etc.)",Game or graphics developer,Educator or academic researcher,...,Gay or Lesbian,Straight or heterosexual,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,Middle Eastern,South Asian,East Asian,"Native American, Pacific Islander, or Indigenous Australian",Salary
0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,10800.0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,73433.0
2,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,66672.0
3,1,0,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,31848.0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0.0


In [81]:
for col in multi_choice_dataset_train.columns:
    print('Start column: '+col)
    name='Gen_map_'+col
    avgname=col+'_avg_sal'
    minname=col+'_min_sal'
    maxname=col+'_max_sal'
    Mapname='Map_'+col
    Salmapname='Salmap_'+col
    name=multi_choice_dataset_train_generate[multi_choice_dataset_train_generate['Salary']>0]
    name[avgname] = name['Salary'].groupby(name[col]).transform('mean')
    name[minname] = name['Salary'].groupby(name[col]).transform('min')
    name[maxname] = name['Salary'].groupby(name[col]).transform('max')
    #Create map for YearsCodingProf
    Mapname=name[[col,avgname,minname,maxname]]
    Salmapname=Mapname.drop_duplicates()
    #print(Salmapname)
    #Mapping new gen features to train, test base on YearsCodingProf
    multi_choice_dataset_train=pd.merge(multi_choice_dataset_train,Salmapname,on=col,how='left')
    multi_choice_dataset_test=pd.merge(multi_choice_dataset_test,Salmapname,on=col,how='left')

Start column: Embedded applications or devices developer


c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarni

Start column: QA or test developer
Start column: DevOps specialist
Start column: Data scientist or machine learning specialist
Start column: Full-stack developer
Start column: Product manager
Start column: Front-end developer
Start column: C-suite executive (CEO, CTO, etc.)
Start column: Game or graphics developer
Start column: Educator or academic researcher
Start column: Desktop or enterprise applications developer
Start column: System administrator
Start column: Student
Start column: Back-end developer
Start column: Marketing or sales professional
Start column: Database administrator
Start column: Designer
Start column: Data or business analyst
Start column: Mobile developer
Start column: Engineering manager
Start column: Facebook
Start column: Stack Overflow Enterprise
Start column: Office / productivity suite (Microsoft Office, Google Suite, etc.)
Start column: Google Hangouts/Chat
Start column: Other chat system (IRC, proprietary software, etc.)
Start column: Slack
Start column: 

Start column: NetBeans
Start column: TextMate
Start column: RubyMine
Start column: Coda
Start column: Vim
Start column: IntelliJ
Start column: Agile
Start column: Pair programming
Start column: Scrum
Start column: Evidence-based software engineering
Start column: Formal standard such as ISO 9001 or IEEE 12207 (aka “waterfall” methodologies)
Start column: Mob programming
Start column: Kanban
Start column: Extreme programming (XP)
Start column: Lean
Start column: PRINCE2
Start column: Copying and pasting files to network shares
Start column: Mercurial
Start column: Team Foundation Version Control
Start column: Git
Start column: Subversion
Start column: I don't use version control
Start column: Zip file back-ups
Start column: The website I was visiting has interesting ads
Start column: The website I was visiting forced me to disable it to access their content
Start column: The website I was visiting asked me to disable it
Start column: The ad-blocking software was causing display issues o

In [82]:
#multi_choice_dataset_train= multi_choice_dataset_train.drop('Salary',axis=1)
print('Training multiple-choices dataset shape: ', multi_choice_dataset_train.shape)
print('Testing multiple-choices dataset shape: ', multi_choice_dataset_test.shape)

Training multiple-choices dataset shape:  (33076, 1280)
Testing multiple-choices dataset shape:  (11259, 1280)


### Categorical "option-choices" columns type

In [83]:
option_choices_cols_name=[col for col in categorical_cols_name if col not in mul_choices_cols_name]

In [84]:
#impute missing value by majority class
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
train_data_arr = imputer.fit_transform(train_data[option_choices_cols_name])
test_data_arr = imputer.transform(test_data[option_choices_cols_name])

In [85]:
option_choices_dataset_train = pd.DataFrame(train_data_arr,columns=option_choices_cols_name)
option_choices_dataset_test = pd.DataFrame(test_data_arr,columns=option_choices_cols_name)

#### Encode option-choices to one-hot columns

In [86]:
# Create a label encoder for option columns
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in option_choices_cols_name:
    # If 2 or fewer unique categories
    if len(list(option_choices_dataset_train[col].unique())) <= 2:
        # Train on the training data
        le.fit(option_choices_dataset_train[col])
        # Transform both training and testing data
        option_choices_dataset_train[col] = le.transform(option_choices_dataset_train[col])
        option_choices_dataset_test[col] = le.transform(option_choices_dataset_test[col])    
        # Keep track of how many columns were label encoded
        le_count += 1
            
print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [87]:
#one-hot encode for train_data
option_choices_dataset_train = pd.get_dummies(option_choices_dataset_train)
option_choices_dataset_train.shape

(33076, 587)

In [88]:
#one-hot encode for test_data
option_choices_dataset_test = pd.get_dummies(option_choices_dataset_test)
option_choices_dataset_test.shape

(11259, 547)

In [89]:
#Align the training and testing data, keep only columns present in both dataframes
option_choices_dataset_train, option_choices_dataset_test = option_choices_dataset_train.align(option_choices_dataset_test, join = 'inner', axis = 1)
print('Training option-choices dataset shape: ', option_choices_dataset_train.shape)
print('Testing option-choices dataset shape: ', option_choices_dataset_test.shape)

Training option-choices dataset shape:  (33076, 537)
Testing option-choices dataset shape:  (11259, 537)


### Combine all dataframes

In [91]:
#float_dataset_train,
train_dataset = pd.concat([float_dataset_train,multi_choice_dataset_train,option_choices_dataset_train],axis=1)

In [92]:
#float_dataset_test,
test_dataset = pd.concat([float_dataset_test,multi_choice_dataset_test,option_choices_dataset_test],axis=1)

In [93]:
for col in train_dataset.columns:
    print (col)
    namecol='squared_of_'+col
    train_dataset[namecol]=train_dataset[col]**2

AssessJob1
AssessJob2
AssessJob3
AssessJob4
AssessJob5
AssessJob6
AssessJob7
AssessJob8
AssessJob9
AssessJob10
AssessBenefits1
AssessBenefits2
AssessBenefits3
AssessBenefits4
AssessBenefits5
AssessBenefits6
AssessBenefits7
AssessBenefits8
AssessBenefits9
AssessBenefits10
AssessBenefits11
JobContactPriorities1
JobContactPriorities2
JobContactPriorities3
JobContactPriorities4
JobContactPriorities5
JobEmailPriorities1
JobEmailPriorities2
JobEmailPriorities3
JobEmailPriorities4
JobEmailPriorities5
JobEmailPriorities6
JobEmailPriorities7
AdsPriorities1
AdsPriorities2
AdsPriorities3
AdsPriorities4
AdsPriorities5
AdsPriorities6
AdsPriorities7
avg_sal
min_sal
max_sal
count_sal
mode_sal
std_sal
YearsCoding_avg_sal
YearsCoding_min_sal
YearsCoding_max_sal
YearsCoding_count_sal
YearsCoding_std_sal
RaceEthnicity_avg_sal
RaceEthnicity_min_sal
RaceEthnicity_max_sal
RaceEthnicity_count_sal
RaceEthnicity_std_sal
YearsCodingProf_avg_sal
YearsCodingProf_min_sal
YearsCodingProf_max_sal
YearsCodingProf_std

SQLite_1
Apache Hive_1
Memcached_1
Drupal
Amazon Echo
Linux
Arduino
Heroku
Android
AWS
Firebase
Google Home
Predix
Azure
iOS
Google Cloud Platform/App Engine
Raspberry Pi
Windows Phone
Serverless
SharePoint
Salesforce
Windows Desktop or Server
ESP 8266
WordPress
Gaming console
Mac OS
Mainframe
IBM Cloud or Watson
Apple Watch or Apple TV
ESP8266
Amazon Echo_1
Drupal_1
Linux_1
Arduino_1
Heroku_1
Android_1
AWS_1
Firebase_1
Google Home_1
Predix_1
Azure_1
iOS_1
Google Cloud Platform/App Engine_1
Raspberry Pi_1
Windows Phone_1
Serverless_1
SharePoint_1
Salesforce_1
Windows Desktop or Server_1
ESP 8266_1
WordPress_1
Gaming console_1
Mac OS_1
Mainframe_1
IBM Cloud or Watson_1
Apple Watch or Apple TV_1
ESP8266_1
Torch/PyTorch
Hadoop
.NET Core
Django
React
Angular
Cordova
Spring
Spark
TensorFlow
Xamarin
Node.js
Torch/PyTorch_1
Hadoop_1
.NET Core_1
Django_1
Angular_1
React_1
Spark_1
Spring_1
Cordova_1
TensorFlow_1
Xamarin_1
Node.js_1
Zend
PHPStorm
Sublime Text
Komodo
Android Studio
Xcode
PyCharm


Internal Wikis, chat rooms, or documentation set up by my company for employees_max_sal
Questions & answers on Stack Overflow_avg_sal
Questions & answers on Stack Overflow_min_sal
Questions & answers on Stack Overflow_max_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_avg_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_min_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_max_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_avg_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_min_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_max_sal
The official documentation and/or standards for the technology_avg_sal
The official documentation and/or standards for the technology_min_sal
The official documentation and/or standards for the technology_max_sal
Online developer communities other than Stack Overflow (ex. forums, listservs, IRC channels, etc.)_avg_sal
O

Drupal_min_sal
Drupal_max_sal
Amazon Echo_avg_sal
Amazon Echo_min_sal
Amazon Echo_max_sal
Linux_avg_sal
Linux_min_sal
Linux_max_sal
Arduino_avg_sal
Arduino_min_sal
Arduino_max_sal
Heroku_avg_sal
Heroku_min_sal
Heroku_max_sal
Android_avg_sal
Android_min_sal
Android_max_sal
AWS_avg_sal
AWS_min_sal
AWS_max_sal
Firebase_avg_sal
Firebase_min_sal
Firebase_max_sal
Google Home_avg_sal
Google Home_min_sal
Google Home_max_sal
Predix_avg_sal
Predix_min_sal
Predix_max_sal
Azure_avg_sal
Azure_min_sal
Azure_max_sal
iOS_avg_sal
iOS_min_sal
iOS_max_sal
Google Cloud Platform/App Engine_avg_sal
Google Cloud Platform/App Engine_min_sal
Google Cloud Platform/App Engine_max_sal
Raspberry Pi_avg_sal
Raspberry Pi_min_sal
Raspberry Pi_max_sal
Windows Phone_avg_sal
Windows Phone_min_sal
Windows Phone_max_sal
Serverless_avg_sal
Serverless_min_sal
Serverless_max_sal
SharePoint_avg_sal
SharePoint_min_sal
SharePoint_max_sal
Salesforce_avg_sal
Salesforce_min_sal
Salesforce_max_sal
Windows Desktop or Server_avg_sal


Female_avg_sal
Female_min_sal
Female_max_sal
Transgender_avg_sal
Transgender_min_sal
Transgender_max_sal
Non-binary, genderqueer, or gender non-conforming_avg_sal
Non-binary, genderqueer, or gender non-conforming_min_sal
Non-binary, genderqueer, or gender non-conforming_max_sal
Male_avg_sal
Male_min_sal
Male_max_sal
Asexual_avg_sal
Asexual_min_sal
Asexual_max_sal
Bisexual or Queer_avg_sal
Bisexual or Queer_min_sal
Bisexual or Queer_max_sal
Gay or Lesbian_avg_sal
Gay or Lesbian_min_sal
Gay or Lesbian_max_sal
Straight or heterosexual_avg_sal
Straight or heterosexual_min_sal
Straight or heterosexual_max_sal
Hispanic or Latino/Latina_avg_sal
Hispanic or Latino/Latina_min_sal
Hispanic or Latino/Latina_max_sal
White or of European descent_avg_sal
White or of European descent_min_sal
White or of European descent_max_sal
Black or of African descent_avg_sal
Black or of African descent_min_sal
Black or of African descent_max_sal
Middle Eastern_avg_sal
Middle Eastern_min_sal
Middle Eastern_max_sa

CurrencySymbol_TTD
CurrencySymbol_TWD
CurrencySymbol_TZS
CurrencySymbol_UAH
CurrencySymbol_UGX
CurrencySymbol_USD
CurrencySymbol_UYU
CurrencySymbol_VND
CurrencySymbol_ZAR
TimeFullyProductive_Less than a month
TimeFullyProductive_More than a year
TimeFullyProductive_Nine months to a year
TimeFullyProductive_One to three months
TimeFullyProductive_Six to nine months
TimeFullyProductive_Three to six months
AgreeDisagree1_Agree
AgreeDisagree1_Disagree
AgreeDisagree1_Neither Agree nor Disagree
AgreeDisagree1_Strongly agree
AgreeDisagree1_Strongly disagree
AgreeDisagree2_Agree
AgreeDisagree2_Disagree
AgreeDisagree2_Neither Agree nor Disagree
AgreeDisagree2_Strongly agree
AgreeDisagree2_Strongly disagree
AgreeDisagree3_Agree
AgreeDisagree3_Disagree
AgreeDisagree3_Neither Agree nor Disagree
AgreeDisagree3_Strongly agree
AgreeDisagree3_Strongly disagree
OperatingSystem_BSD/Unix
OperatingSystem_Linux-based
OperatingSystem_MacOS
OperatingSystem_Windows
NumberMonitors_1
NumberMonitors_2
NumberMoni

In [94]:
for col in test_dataset.columns:
    print (col)
    namecol='squared_of_'+col
    test_dataset[namecol]=test_dataset[col]**2

AssessJob1
AssessJob2
AssessJob3
AssessJob4
AssessJob5
AssessJob6
AssessJob7
AssessJob8
AssessJob9
AssessJob10
AssessBenefits1
AssessBenefits2
AssessBenefits3
AssessBenefits4
AssessBenefits5
AssessBenefits6
AssessBenefits7
AssessBenefits8
AssessBenefits9
AssessBenefits10
AssessBenefits11
JobContactPriorities1
JobContactPriorities2
JobContactPriorities3
JobContactPriorities4
JobContactPriorities5
JobEmailPriorities1
JobEmailPriorities2
JobEmailPriorities3
JobEmailPriorities4
JobEmailPriorities5
JobEmailPriorities6
JobEmailPriorities7
AdsPriorities1
AdsPriorities2
AdsPriorities3
AdsPriorities4
AdsPriorities5
AdsPriorities6
AdsPriorities7
avg_sal
min_sal
max_sal
count_sal
mode_sal
std_sal
YearsCoding_avg_sal
YearsCoding_min_sal
YearsCoding_max_sal
YearsCoding_count_sal
YearsCoding_std_sal
RaceEthnicity_avg_sal
RaceEthnicity_min_sal
RaceEthnicity_max_sal
RaceEthnicity_count_sal
RaceEthnicity_std_sal
YearsCodingProf_avg_sal
YearsCodingProf_min_sal
YearsCodingProf_max_sal
YearsCodingProf_std

Salesforce_1
Windows Desktop or Server_1
ESP 8266_1
WordPress_1
Gaming console_1
Mac OS_1
Mainframe_1
IBM Cloud or Watson_1
Apple Watch or Apple TV_1
ESP8266_1
Torch/PyTorch
Hadoop
.NET Core
Django
React
Angular
Cordova
Spring
Spark
TensorFlow
Xamarin
Node.js
Torch/PyTorch_1
Hadoop_1
.NET Core_1
Django_1
Angular_1
React_1
Spark_1
Spring_1
Cordova_1
TensorFlow_1
Xamarin_1
Node.js_1
Zend
PHPStorm
Sublime Text
Komodo
Android Studio
Xcode
PyCharm
Eclipse
Atom
IPython / Jupyter
Emacs
RStudio
Visual Studio
Visual Studio Code
Light Table
Notepad++
NetBeans
TextMate
RubyMine
Coda
Vim
IntelliJ
Agile
Pair programming
Scrum
Evidence-based software engineering
Formal standard such as ISO 9001 or IEEE 12207 (aka “waterfall” methodologies)
Mob programming
Kanban
Extreme programming (XP)
Lean
PRINCE2
Copying and pasting files to network shares
Mercurial
Team Foundation Version Control
Git
Subversion
I don't use version control
Zip file back-ups
The website I was visiting has interesting ads
The websi

Perl_min_sal
Perl_max_sal
Lua_avg_sal
Lua_min_sal
Lua_max_sal
R_avg_sal
R_min_sal
R_max_sal
Hack_avg_sal
Hack_min_sal
Hack_max_sal
VB.NET_avg_sal
VB.NET_min_sal
VB.NET_max_sal
JavaScript_avg_sal
JavaScript_min_sal
JavaScript_max_sal
C++_avg_sal
C++_min_sal
C++_max_sal
Assembly_avg_sal
Assembly_min_sal
Assembly_max_sal
Objective-C_avg_sal
Objective-C_min_sal
Objective-C_max_sal
Kotlin_avg_sal
Kotlin_min_sal
Kotlin_max_sal
Java_avg_sal
Java_min_sal
Java_max_sal
HTML_avg_sal
HTML_min_sal
HTML_max_sal
SQL_avg_sal
SQL_min_sal
SQL_max_sal
Julia_avg_sal
Julia_min_sal
Julia_max_sal
C#_avg_sal
C#_min_sal
C#_max_sal
Python_avg_sal
Python_min_sal
Python_max_sal
Rust_avg_sal
Rust_min_sal
Rust_max_sal
Ocaml_avg_sal
Ocaml_min_sal
Ocaml_max_sal
Swift_1_avg_sal
Swift_1_min_sal
Swift_1_max_sal
Ruby_1_avg_sal
Ruby_1_min_sal
Ruby_1_max_sal
Scala_1_avg_sal
Scala_1_min_sal
Scala_1_max_sal
CoffeeScript_1_avg_sal
CoffeeScript_1_min_sal
CoffeeScript_1_max_sal
Bash/Shell_1_avg_sal
Bash/Shell_1_min_sal
Bash/She

Node.js_max_sal
Torch/PyTorch_1_avg_sal
Torch/PyTorch_1_min_sal
Torch/PyTorch_1_max_sal
Hadoop_1_avg_sal
Hadoop_1_min_sal
Hadoop_1_max_sal
.NET Core_1_avg_sal
.NET Core_1_min_sal
.NET Core_1_max_sal
Django_1_avg_sal
Django_1_min_sal
Django_1_max_sal
Angular_1_avg_sal
Angular_1_min_sal
Angular_1_max_sal
React_1_avg_sal
React_1_min_sal
React_1_max_sal
Spark_1_avg_sal
Spark_1_min_sal
Spark_1_max_sal
Spring_1_avg_sal
Spring_1_min_sal
Spring_1_max_sal
Cordova_1_avg_sal
Cordova_1_min_sal
Cordova_1_max_sal
TensorFlow_1_avg_sal
TensorFlow_1_min_sal
TensorFlow_1_max_sal
Xamarin_1_avg_sal
Xamarin_1_min_sal
Xamarin_1_max_sal
Node.js_1_avg_sal
Node.js_1_min_sal
Node.js_1_max_sal
Zend_avg_sal
Zend_min_sal
Zend_max_sal
PHPStorm_avg_sal
PHPStorm_min_sal
PHPStorm_max_sal
Sublime Text_avg_sal
Sublime Text_min_sal
Sublime Text_max_sal
Komodo_avg_sal
Komodo_min_sal
Komodo_max_sal
Android Studio_avg_sal
Android Studio_min_sal
Android Studio_max_sal
Xcode_avg_sal
Xcode_min_sal
Xcode_max_sal
PyCharm_avg_sal

HopeFiveYears_Retirement
HopeFiveYears_Working as a founder or co-founder of my own company
HopeFiveYears_Working as a product manager or project manager
HopeFiveYears_Working as an engineering manager or other functional manager
HopeFiveYears_Working in a career completely unrelated to software development
HopeFiveYears_Working in a different or more specialized technical role than the one I'm in now
JobSearchStatus_I am actively looking for a job
JobSearchStatus_I am not interested in new job opportunities
JobSearchStatus_I’m not actively looking, but I am open to new opportunities
LastNewJob_Between 1 and 2 years ago
LastNewJob_Between 2 and 4 years ago
LastNewJob_I've never had a job
LastNewJob_Less than a year ago
LastNewJob_More than 4 years ago
UpdateCV_A friend told me about a job opportunity
UpdateCV_A recruiter contacted me
UpdateCV_I did not receive an expected change in compensation
UpdateCV_I had a negative experience or interaction at work
UpdateCV_I received bad news abo

In [95]:
print('Training dataset shape: ', train_dataset.shape)
#print('Training label shape: ', y_train.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (33076, 4100)
Testing dataset shape:  (11259, 4100)


# 2. TRAINING

In [96]:
from sklearn.utils.validation import column_or_1d

In [97]:
X_train= train_dataset
y_train= target.astype(int)
X_test= test_dataset

In [98]:
y_train.dtype

dtype('int32')

In [99]:
import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

### Split training set to validation set

In [100]:
params = {
    'objective'         : 'regression',    
    'metric'            : 'l2', 
    'nthread'           : 4,
    'learning_rate'     : 0.001,

    'num_leaves'        : 23,
    'feature_fraction'  : 0.106,
    'bagging_fraction'  : 0.825,
    'max_depth'         : -1,
    'lambda_l1'         : 0.1,
    'lambda_l2'         : 2.5,
    'min_split_gain'    : 0.007,
}

# # Create parameters to search
# gridParams = {
#     'lambda_l1' : [0.1,0.2,0.3],
#     'lambda_l2' : [2.5,2.7,2.9],
#     }

# # Create REGRESSOR to use. Note that parameters have to be input manually
# # not as a dict!
# mdl = lgb.LGBMRegressor(
#           objective = params['objective'],
#           metric = params['metric'],
#           nthread = params['nthread'],
#           learning_rate = params['learning_rate'],
#           num_leaves = params['num_leaves'],
#           feature_fraction = params['feature_fraction'],
#           bagging_fraction = params['bagging_fraction'],
#           max_depth = params['max_depth'],
#           lambda_l1 = params['lambda_l1'],
#           lambda_l2 = params['lambda_l2'],
#           min_split_gain = params['min_split_gain'])

# # To view the default model params:
# mdl.get_params().keys()

# # Create the grid
# grid = GridSearchCV(mdl, gridParams,
#                     verbose=0,
#                     cv=4,
#                     n_jobs=2)

# # Run the grid
# grid.fit(X_train, y_train)

# # Print the best parameters found
# print(grid.best_params_)
# print(grid.best_score_)

# # Using parameters already set above, replace in the best from the grid search
# params['lambda_l1'] = grid.best_params_['lambda_l1']
# params['lambda_l2'] = grid.best_params_['lambda_l2']

# print('Fitting with params: ')
# print(params)

In [101]:
skf = StratifiedKFold(n_splits=4, random_state=3462873, shuffle=True)
preds = 0.0
for itrain, ivalid in skf.split(X_train, y_train):
    lgb_train = lgb.Dataset(X_train.iloc[itrain], y_train.iloc[itrain])
    lgb_eval  = lgb.Dataset(X_train.iloc[ivalid], y_train.iloc[ivalid], reference = lgb_train)
    model = lgb.train(params,
                lgb_train,
                num_boost_round = 999999,  
                early_stopping_rounds = 800,
                verbose_eval = 100,
                valid_sets = [lgb_train, lgb_eval])
    pred = model.predict(X_test)
    preds += pred/skf.n_splits

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


Training until validation scores don't improve for 800 rounds
[100]	training's l2: 1.74247e+09	valid_1's l2: 1.71803e+09
[200]	training's l2: 1.55881e+09	valid_1's l2: 1.5352e+09
[300]	training's l2: 1.40287e+09	valid_1's l2: 1.38021e+09
[400]	training's l2: 1.27112e+09	valid_1's l2: 1.24954e+09
[500]	training's l2: 1.16134e+09	valid_1's l2: 1.14071e+09
[600]	training's l2: 1.06859e+09	valid_1's l2: 1.049e+09
[700]	training's l2: 9.90184e+08	valid_1's l2: 9.71515e+08
[800]	training's l2: 9.2295e+08	valid_1's l2: 9.05372e+08
[900]	training's l2: 8.66074e+08	valid_1's l2: 8.49719e+08
[1000]	training's l2: 8.17416e+08	valid_1's l2: 8.02303e+08
[1100]	training's l2: 7.75638e+08	valid_1's l2: 7.61656e+08
[1200]	training's l2: 7.39616e+08	valid_1's l2: 7.26743e+08
[1300]	training's l2: 7.08244e+08	valid_1's l2: 6.96512e+08
[1400]	training's l2: 6.81234e+08	valid_1's l2: 6.70561e+08
[1500]	training's l2: 6.57501e+08	valid_1's l2: 6.47855e+08
[1600]	training's l2: 6.36546e+08	valid_1's l2: 6.2

[13700]	training's l2: 3.21593e+08	valid_1's l2: 4.05767e+08
[13800]	training's l2: 3.20898e+08	valid_1's l2: 4.05639e+08
[13900]	training's l2: 3.20222e+08	valid_1's l2: 4.05514e+08
[14000]	training's l2: 3.19532e+08	valid_1's l2: 4.05379e+08
[14100]	training's l2: 3.1885e+08	valid_1's l2: 4.05243e+08
[14200]	training's l2: 3.18163e+08	valid_1's l2: 4.05128e+08
[14300]	training's l2: 3.17482e+08	valid_1's l2: 4.05003e+08
[14400]	training's l2: 3.16822e+08	valid_1's l2: 4.04888e+08
[14500]	training's l2: 3.16159e+08	valid_1's l2: 4.04777e+08
[14600]	training's l2: 3.15515e+08	valid_1's l2: 4.04654e+08
[14700]	training's l2: 3.14859e+08	valid_1's l2: 4.04558e+08
[14800]	training's l2: 3.14209e+08	valid_1's l2: 4.04448e+08
[14900]	training's l2: 3.13568e+08	valid_1's l2: 4.04347e+08
[15000]	training's l2: 3.12914e+08	valid_1's l2: 4.04246e+08
[15100]	training's l2: 3.12277e+08	valid_1's l2: 4.04182e+08
[15200]	training's l2: 3.11651e+08	valid_1's l2: 4.04093e+08
[15300]	training's l2: 3.

[27200]	training's l2: 2.54998e+08	valid_1's l2: 3.98132e+08
[27300]	training's l2: 2.5462e+08	valid_1's l2: 3.98101e+08
[27400]	training's l2: 2.54238e+08	valid_1's l2: 3.98065e+08
[27500]	training's l2: 2.53855e+08	valid_1's l2: 3.98033e+08
[27600]	training's l2: 2.53475e+08	valid_1's l2: 3.98009e+08
[27700]	training's l2: 2.53097e+08	valid_1's l2: 3.97985e+08
[27800]	training's l2: 2.52724e+08	valid_1's l2: 3.97962e+08
[27900]	training's l2: 2.52359e+08	valid_1's l2: 3.97937e+08
[28000]	training's l2: 2.51978e+08	valid_1's l2: 3.97902e+08
[28100]	training's l2: 2.51605e+08	valid_1's l2: 3.97881e+08
[28200]	training's l2: 2.51244e+08	valid_1's l2: 3.97863e+08
[28300]	training's l2: 2.50872e+08	valid_1's l2: 3.9783e+08
[28400]	training's l2: 2.50514e+08	valid_1's l2: 3.97798e+08
[28500]	training's l2: 2.50136e+08	valid_1's l2: 3.9777e+08
[28600]	training's l2: 2.4977e+08	valid_1's l2: 3.9773e+08
[28700]	training's l2: 2.49407e+08	valid_1's l2: 3.97704e+08
[28800]	training's l2: 2.4904

[40700]	training's l2: 2.10684e+08	valid_1's l2: 3.95135e+08
[40800]	training's l2: 2.10402e+08	valid_1's l2: 3.9512e+08
[40900]	training's l2: 2.10115e+08	valid_1's l2: 3.95105e+08
[41000]	training's l2: 2.09829e+08	valid_1's l2: 3.95091e+08
[41100]	training's l2: 2.09541e+08	valid_1's l2: 3.9508e+08
[41200]	training's l2: 2.09256e+08	valid_1's l2: 3.9507e+08
[41300]	training's l2: 2.0897e+08	valid_1's l2: 3.95054e+08
[41400]	training's l2: 2.08696e+08	valid_1's l2: 3.95039e+08
[41500]	training's l2: 2.08414e+08	valid_1's l2: 3.95021e+08
[41600]	training's l2: 2.08138e+08	valid_1's l2: 3.95009e+08
[41700]	training's l2: 2.07861e+08	valid_1's l2: 3.94991e+08
[41800]	training's l2: 2.07577e+08	valid_1's l2: 3.94981e+08
[41900]	training's l2: 2.07293e+08	valid_1's l2: 3.94957e+08
[42000]	training's l2: 2.07017e+08	valid_1's l2: 3.94939e+08
[42100]	training's l2: 2.06734e+08	valid_1's l2: 3.94931e+08
[42200]	training's l2: 2.06456e+08	valid_1's l2: 3.94909e+08
[42300]	training's l2: 2.061

...

[63600]	training's l2: 1.53053e+08	valid_1's l2: 4.20429e+08
[63700]	training's l2: 1.52873e+08	valid_1's l2: 4.20427e+08
[63800]	training's l2: 1.52689e+08	valid_1's l2: 4.20423e+08
[63900]	training's l2: 1.5251e+08	valid_1's l2: 4.20422e+08
[64000]	training's l2: 1.52328e+08	valid_1's l2: 4.20416e+08
[64100]	training's l2: 1.52147e+08	valid_1's l2: 4.20412e+08
[64200]	training's l2: 1.51965e+08	valid_1's l2: 4.20396e+08
[64300]	training's l2: 1.51776e+08	valid_1's l2: 4.20388e+08
[64400]	training's l2: 1.51592e+08	valid_1's l2: 4.20384e+08
[64500]	training's l2: 1.51418e+08	valid_1's l2: 4.20371e+08
[64600]	training's l2: 1.51229e+08	valid_1's l2: 4.2037e+08
[64700]	training's l2: 1.5105e+08	valid_1's l2: 4.20361e+08
[64800]	training's l2: 1.50873e+08	valid_1's l2: 4.20352e+08
[64900]	training's l2: 1.5069e+08	valid_1's l2: 4.20339e+08
[65000]	training's l2: 1.50508e+08	valid_1's l2: 4.20328e+08
[65100]	training's l2: 1.50337e+08	valid_1's l2: 4.20328e+08
[65200]	training's l2: 1.501

[77100]	training's l2: 1.30716e+08	valid_1's l2: 4.19445e+08
[77200]	training's l2: 1.3056e+08	valid_1's l2: 4.19438e+08
[77300]	training's l2: 1.30404e+08	valid_1's l2: 4.19429e+08
[77400]	training's l2: 1.30258e+08	valid_1's l2: 4.19417e+08
[77500]	training's l2: 1.30116e+08	valid_1's l2: 4.19413e+08
[77600]	training's l2: 1.29968e+08	valid_1's l2: 4.19397e+08
[77700]	training's l2: 1.29814e+08	valid_1's l2: 4.19394e+08
[77800]	training's l2: 1.29667e+08	valid_1's l2: 4.19391e+08
[77900]	training's l2: 1.2952e+08	valid_1's l2: 4.19387e+08
[78000]	training's l2: 1.29368e+08	valid_1's l2: 4.19377e+08
[78100]	training's l2: 1.29217e+08	valid_1's l2: 4.19366e+08
[78200]	training's l2: 1.29074e+08	valid_1's l2: 4.19357e+08
[78300]	training's l2: 1.28926e+08	valid_1's l2: 4.19351e+08
[78400]	training's l2: 1.2878e+08	valid_1's l2: 4.19348e+08
[78500]	training's l2: 1.28635e+08	valid_1's l2: 4.19348e+08
[78600]	training's l2: 1.28489e+08	valid_1's l2: 4.19335e+08
[78700]	training's l2: 1.28

[90600]	training's l2: 1.11872e+08	valid_1's l2: 4.18731e+08
[90700]	training's l2: 1.11744e+08	valid_1's l2: 4.18729e+08
[90800]	training's l2: 1.11623e+08	valid_1's l2: 4.18725e+08
[90900]	training's l2: 1.11497e+08	valid_1's l2: 4.18721e+08
[91000]	training's l2: 1.1137e+08	valid_1's l2: 4.18715e+08
[91100]	training's l2: 1.11241e+08	valid_1's l2: 4.18719e+08
[91200]	training's l2: 1.11114e+08	valid_1's l2: 4.1872e+08
[91300]	training's l2: 1.10988e+08	valid_1's l2: 4.18717e+08
[91400]	training's l2: 1.10865e+08	valid_1's l2: 4.18718e+08
[91500]	training's l2: 1.10739e+08	valid_1's l2: 4.18712e+08
[91600]	training's l2: 1.10616e+08	valid_1's l2: 4.18711e+08
[91700]	training's l2: 1.10489e+08	valid_1's l2: 4.18702e+08
[91800]	training's l2: 1.10362e+08	valid_1's l2: 4.18699e+08
[91900]	training's l2: 1.10231e+08	valid_1's l2: 4.18691e+08
[92000]	training's l2: 1.10109e+08	valid_1's l2: 4.18686e+08
[92100]	training's l2: 1.09979e+08	valid_1's l2: 4.18673e+08
[92200]	training's l2: 1.0

[104000]	training's l2: 9.60237e+07	valid_1's l2: 4.18259e+08
[104100]	training's l2: 9.59253e+07	valid_1's l2: 4.1826e+08
[104200]	training's l2: 9.58237e+07	valid_1's l2: 4.18256e+08
[104300]	training's l2: 9.57223e+07	valid_1's l2: 4.1825e+08
[104400]	training's l2: 9.56148e+07	valid_1's l2: 4.18249e+08
[104500]	training's l2: 9.5507e+07	valid_1's l2: 4.18243e+08
[104600]	training's l2: 9.53999e+07	valid_1's l2: 4.1824e+08
[104700]	training's l2: 9.52936e+07	valid_1's l2: 4.18234e+08
[104800]	training's l2: 9.51824e+07	valid_1's l2: 4.1823e+08
[104900]	training's l2: 9.50817e+07	valid_1's l2: 4.18224e+08
[105000]	training's l2: 9.4978e+07	valid_1's l2: 4.18221e+08
[105100]	training's l2: 9.48757e+07	valid_1's l2: 4.1822e+08
[105200]	training's l2: 9.47694e+07	valid_1's l2: 4.18216e+08
[105300]	training's l2: 9.46626e+07	valid_1's l2: 4.18207e+08
[105400]	training's l2: 9.456e+07	valid_1's l2: 4.18201e+08
[105500]	training's l2: 9.44585e+07	valid_1's l2: 4.18197e+08
[105600]	training

[117300]	training's l2: 8.26944e+07	valid_1's l2: 4.17885e+08
[117400]	training's l2: 8.26007e+07	valid_1's l2: 4.17884e+08
[117500]	training's l2: 8.25103e+07	valid_1's l2: 4.1788e+08
[117600]	training's l2: 8.24167e+07	valid_1's l2: 4.17879e+08
[117700]	training's l2: 8.23265e+07	valid_1's l2: 4.17882e+08
[117800]	training's l2: 8.2232e+07	valid_1's l2: 4.1788e+08
[117900]	training's l2: 8.21317e+07	valid_1's l2: 4.17863e+08
[118000]	training's l2: 8.2034e+07	valid_1's l2: 4.17859e+08
[118100]	training's l2: 8.19358e+07	valid_1's l2: 4.17853e+08
[118200]	training's l2: 8.18412e+07	valid_1's l2: 4.17847e+08
[118300]	training's l2: 8.17509e+07	valid_1's l2: 4.17853e+08
[118400]	training's l2: 8.16631e+07	valid_1's l2: 4.17848e+08
[118500]	training's l2: 8.15759e+07	valid_1's l2: 4.17847e+08
[118600]	training's l2: 8.14831e+07	valid_1's l2: 4.17848e+08
[118700]	training's l2: 8.13945e+07	valid_1's l2: 4.17848e+08
[118800]	training's l2: 8.13062e+07	valid_1's l2: 4.17847e+08
[118900]	tra

# 3. PREDICTING

In [110]:
def to_submit_file(file_name, values):
    submit = test_id.copy()
    submit['Salary'] = values
    submit.loc[submit['Salary'] < 0] = 0
    submit.to_csv(file_name, index = False)
    print ('Done writting submit file: ', file_name)

In [111]:
test_id = pd.read_csv('data_all/submit.csv')

In [112]:
y_pred = preds

In [113]:
to_submit_file('submitfile.csv',y_pred)

Done writting submit file:  submitfile.csv
